In [373]:
import pandas as pd

# read source file snapshots
df = pd.read_excel(r'G:\DM_AutoWrite\l_compare_snapshots_20230224_Test.xlsx', sheet_name = 'differences')

# read als file
df1 = pd.read_excel(r'G:\DM_AutoWrite\BGB-A317-306_V9.0_CC7_9Dec20_1053.xlsx',sheet_name='Fields')


In [374]:
pd.set_option('display.max_rows', 1)
pd.set_option('display.max_columns', 30)

In [375]:
df

,Project,Site Number,Subject,Instance Name,Data Set Name,CRF Page Name,Record Id,Record Position,Lab Form,Analyte Name,Variable Name,Variable Label,Previous Value,Current Value,Status,Record Date,Cut off Date,Newflag,Comments,Closed?
0,BGB-A317-306,34026,034026-003,Unscheduled Visits 17 Sep 2020,ADASAMP,Anti-tislelizumab antibodies,129931798,0,NaN,NaN,ADADAT_RAW,Sample Collection Date (Character),17 SEP 2021,17 SEP 2020,Updated,17 SEP 2020,28Feb2022,Old,YY_7/11/2022: Record in date change tracker,Open


In [376]:
df1

,FormOID,FieldOID,Ordinal,DraftFieldNumber,DraftFieldName,DraftFieldActive,VariableOID,DataFormat,DataDictionaryName,UnitDictionaryName,CodingDictionary,ControlType,AcceptableFileExtensions,IndentLevel,PreText,...,DoesNotBreakSignature,LowerRange,UpperRange,NCLowerRange,NCUpperRange,ViewRestrictions,EntryRestrictions,ReviewGroups,IsVisualVerify,FDownloadedFromObjectId,FSourceObjectId,VDownloadedFromObjectId,VSourceObjectId,FSourceUrlId,VSourceUrlId
0,SUBJECT,SUBJID,1,NaN,SUBJID,True,SUBJID,$15,NaN,NaN,NaN,Text,NaN,0,Subject number,...,False,NaN,NaN,NaN,NaN,NaN,NaN,Data Management,False,7466485,6963714,6699783.0,6247376.0,NaN,NaN


#### Preparation

In [377]:
#### df1 -> df2(FormOID, VariableOID, Ordinal, IsLog)
df2 = pd.DataFrame(df1,columns=['FormOID','DraftFieldName','Ordinal','IsLog'])
df2

,FormOID,DraftFieldName,Ordinal,IsLog
0,SUBJECT,SUBJID,1,False


In [378]:
#### create a ref col'VarOID_Ref', with removing suffixes
def RemoveSuffix(x):
    suffix_list = ['_RAW','_LLT','_HLGT','_HLT','_PT','_SOC','_ATC4','_ATC3','_ATC2','_ATC','_PRODSYN','_PROD']
    action_taken = False
    for item in suffix_list:
        if x.__contains__(item):
            return x.replace(item,' ').strip()
            action_taken = True
    if action_taken == False:
        return x
    
df['VarOID_Ref'] = df.apply(lambda x: RemoveSuffix(x['Variable Name']),axis=1)

df

,Project,Site Number,Subject,Instance Name,Data Set Name,CRF Page Name,Record Id,Record Position,Lab Form,Analyte Name,Variable Name,Variable Label,Previous Value,Current Value,Status,Record Date,Cut off Date,Newflag,Comments,Closed?,VarOID_Ref
0,BGB-A317-306,34026,034026-003,Unscheduled Visits 17 Sep 2020,ADASAMP,Anti-tislelizumab antibodies,129931798,0,NaN,NaN,ADADAT_RAW,Sample Collection Date (Character),17 SEP 2021,17 SEP 2020,Updated,17 SEP 2020,28Feb2022,Old,YY_7/11/2022: Record in date change tracker,Open,ADADAT


In [379]:
#### df2 +-> df
#df = pd.merge(df, df2, how = 'left', left_on=['Data Set Name','Variable Name'], right_on=['FormOID','VariableOID'])
df = pd.merge(df, df2, how = 'left', left_on=['Data Set Name','VarOID_Ref'], right_on=['FormOID','DraftFieldName'])
df

,Project,Site Number,Subject,Instance Name,Data Set Name,CRF Page Name,Record Id,Record Position,Lab Form,Analyte Name,Variable Name,Variable Label,Previous Value,Current Value,Status,Record Date,Cut off Date,Newflag,Comments,Closed?,VarOID_Ref,FormOID,DraftFieldName,Ordinal,IsLog
0,BGB-A317-306,34026,034026-003,Unscheduled Visits 17 Sep 2020,ADASAMP,Anti-tislelizumab antibodies,129931798,0,NaN,NaN,ADADAT_RAW,Sample Collection Date (Character),17 SEP 2021,17 SEP 2020,Updated,17 SEP 2020,28Feb2022,Old,YY_7/11/2022: Record in date change tracker,Open,ADADAT,ADASAMP,ADADAT,2.0,False


In [380]:
#### fill empty value
df.fillna('Blank', inplace = True)

In [381]:
#### outout
#df.to_excel(r'G:\DM_AutoWrite\df.xlsx',index=False)

In [382]:
#### order by ordinal
df = df.sort_values(['Subject','Data Set Name','Record Position','Ordinal'], ascending=[False, False, False, True])
# output
#df.to_excel(r'G:\DM_AutoWrite\df.xlsx',index=False)

In [383]:
# add ref_col 'recPos_Ref'
def UpdateRecPos(x, y):
    if x is True:
        return y
    elif x is False:
        return '0'
    elif x == 'Blank': #99x083x
        return '0'
    
df['RecPos_Ref'] = df.apply(lambda x: UpdateRecPos(x['IsLog'], x['Record Position']),axis=1)

# output
df.to_excel(r'G:\DM_AutoWrite\df.xlsx',index=False)

In [384]:
# Group_by to combine Previous Value together
df_agg =df.groupby(['Subject','Instance Name', 'Data Set Name','Record Id', 'Record Position','RecPos_Ref'])['Previous Value'].apply(lambda x: '\n'.join(x)).reset_index(name='PreVal_AGG')
# output
df_agg.to_excel(r'G:\DM_AutoWrite\df_agg.xlsx',index=False)

In [385]:
# Group_by to combine field label together
df_agg1 =df.groupby(['Subject','Instance Name', 'Data Set Name','Record Id','Record Position','RecPos_Ref'])['Variable Label'].apply(lambda x: '\n'.join(x)).reset_index(name='Label_AGG')
# output
df_agg1.to_excel(r'G:\DM_AutoWrite\df_agg1.xlsx',index=False)

In [386]:
# Group_by to combine Current Value together
df_agg2 =df.groupby(['Subject','Instance Name', 'Data Set Name','Record Id','Record Position','RecPos_Ref'])['Current Value'].apply(lambda x: '\n'.join(x)).reset_index(name='CurVal_AGG')
# output
df_agg2.to_excel(r'G:\DM_AutoWrite\df_agg2.xlsx',index=False)

In [387]:
# Combine df_agg, agg1 and agg2 together
# because df_agg is the prime table, copy its' col 'Label_AGG' since after merging the original one will be offset
#df_agg.loc[:,'Email_Upper'] = df_agg.loc[:,'Email']

df_comb_01 = pd.merge(df_agg, df_agg1, how = 'left', left_on=['Subject','Instance Name','Data Set Name','Record Id','Record Position','RecPos_Ref'], right_on=['Subject','Instance Name','Data Set Name','Record Id','Record Position','RecPos_Ref'])
df_comb_02 = pd.merge(df_comb_01, df_agg2, how = 'left', left_on=['Subject','Instance Name','Data Set Name','Record Id','Record Position','RecPos_Ref'], right_on=['Subject','Instance Name','Data Set Name','Record Id','Record Position','RecPos_Ref'])
# output
df_comb_02.to_excel(r'G:\DM_AutoWrite\df_comb_merge.xlsx',index=False)

In [388]:
# add a sequential column, named as 'Tracker ID'
df_comb_02.insert(0,'Sequence', range(1, 1 + len(df_comb_02)))

In [389]:
# adjust column PreVal_AGG
# add ref_col 'recPos_Ref'
def PreVal_Adj(x):
    lst = x.split('\n')
    count = 0
    for item in lst:
        if item == 'Blank':
            count += 1
    if count == len(lst):
        return 'Blank'
    else:
        return x
     
df_comb_02['PreVal_AGG_Adj'] = df_comb_02.apply(lambda x: PreVal_Adj(x['PreVal_AGG']),axis=1)
# output
df_comb_02.to_excel(r'G:\DM_AutoWrite\df_comb_blankTest.xlsx',index=False)

In [390]:
# add a new column named as 'Visit Name / Form or Dataset Name'
def CombineCols(x, y, z):
    if pd.isna(z):
        return str(x) + ' ' + '/' + ' ' + str(y)
    elif ~pd.isna(z):
        return str(x) + ' ' + '/' + ' ' + str(y) + ' log#' + str(z)
    
df_comb_02['Visit Name/Form or Dataset Name'] = df_comb_02.apply(lambda x: CombineCols(x['Instance Name'], x['Data Set Name'], x['RecPos_Ref']),axis=1)
# output
#df_comb_02.to_excel(r'G:\DM_AutoWrite\df_comb.xlsx',index=False)

In [391]:
# rename columns
#df_comb_02 = df_comb_02.rename(columns={'Subject': 'Subject Number', 'PreVal_AGG': 'Data in Clinical Database'})

# drop columns
df_comb_02 = df_comb_02.drop(columns = ['Instance Name','Data Set Name','Record Position','RecPos_Ref','PreVal_AGG'])

# reorder columns
df_comb_02 = pd.DataFrame(df_comb_02,columns=(['Sequence','Subject','Visit Name/Form or Dataset Name','Label_AGG','PreVal_AGG_Adj','CurVal_AGG']))

# output
df_comb_02.to_excel(r'G:\DM_AutoWrite\df_comb.xlsx',index=False)

#### Copy and Paste

In [392]:
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

# Load workbooks:
wb = openpyxl.load_workbook(r'G:\DM_AutoWrite\df_comb.xlsx', data_only=True)
wb1 = openpyxl.load_workbook(r'G:\DM_AutoWrite\BGB-A317-306 Data Change Tracker_Test1.xlsx') #BGB-A317-306 Data Change Tracker_Test1.xlsx

# define worksheet:
ws = wb['Sheet1'] 
ws1 = wb1['Data Change Tracker']

for row in ws['A2':'F'+ str(ws.max_row)]:
    for cell in row:
        #print(cell.value)
        #print(ws1[cell.coordinate].value)
        ws1[cell.coordinate].value = cell.value
        #print(ws1[cell.coordinate].value)
        
wb1.save('G:\DM_AutoWrite\BGB-A317-306 Data Change Tracker_Test1.xlsx')




# for i in range(2, ws.max_row):
#     for j in range(1, 6):
#         #print (ws.cell(row=i,column=j).value)
#         #print (ws1.cell(row=i,column=j).value)
#         ws1.cell(row=i,column=j).value = 'test'
#         #ws.cell(row=i,column=j).value
        
#wb1.save('G:\DM_AutoWrite\BGB-A317-306 Data Change Tracker_Test1.xlsx')